### Demo Ordini di Servizio Sogei with Cohere


* see: https://python.langchain.com/docs/use_cases/question_answering/

In [1]:
# Document Loader
from langchain.document_loaders import DirectoryLoader

# Text Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Embeddings and Vector Store (Vector DB)
# removed OpenAI, using HF for embeddings
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

from langchain import hub

# removed OpenAI, using Cohere
from langchain.chat_models import ChatCohere

from langchain.schema.runnable import RunnablePassthrough

import oci

from oci_llm import OCIGenAILLM

# private configs
from config_private import COMPARTMENT_OCID

In [2]:
# read OCI config
config = oci.config.from_file("~/.oci/config")

ENDPOINT = "https://generativeai.aiservice.us-chicago-1.oci.oraclecloud.com"

#### Loading the document: a set of local txt documents

In [3]:
LOCAL_DIR = "./docs"
LOCAL_EXT = "*.txt"

loader = DirectoryLoader(LOCAL_DIR, glob=LOCAL_EXT)

data = loader.load()

In [4]:
print(f"We have loaded {len(data)} documents...")

We have loaded 5 documents...


#### Splitting the document in chunks

In [5]:
CHUNK_SIZE = 1024

text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=0)

splits = text_splitter.split_documents(data)

In [6]:
print(f"We have {len(splits)} splits...")

We have 18 splits...


In [16]:
splits[0]

Document(page_content='Pag. 1 di 2 Sogei 23 giugno 2020 ORDINE DI SERVIZIO N. 4 Con decorrenza 1° luglio 2020, la Direzione Finanza Pubblica e altre Convenzioni, la cui responsabilità è affidata ad Alessandra Sbezzi, è rinominata Finanza Pubblica. La Direzione assume il seguente assetto organizzativo: \uf02d Funzione RGS Sistemi ERP, affidata a Gianna Caralla, alla quale riportano le seguenti strutture: \uf0b7 Unità Organizzativa Bilancio Finanziario Responsabile: Francesco Fabiani; \uf0b7 Unità Organizzativa Contabilità Responsabile: Antonella Latella; \uf0b7 Unità Organizzativa Sistemi di Monitoraggio e Vigilanza Responsabile: Grazia Varano; \uf02d Funzione RGS Sistemi Trasversali e Corte dei Conti, affidata a Tiziana Allegretti, alla quale riportano le seguenti strutture: \uf0b7 Unità Organizzativa Corte dei Conti Responsabile: Stefano Ferrari; \uf0b7 Unità Organizzativa Sistemi UE e IGIT Responsabile: Antonella Merola; \uf0b7 Unità Organizzativa Data Management Responsabile: Gianlu

#### Create Vector DB and embeddings

In [8]:
# We have substituted OpenAI with HF
EMBED_MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"

model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": False}


hf = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL_NAME, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

# using Chroma as Vector store
vectorstore = Chroma.from_documents(documents=splits, embedding=hf)

retriever = vectorstore.as_retriever()

#### Define the prompt structure

In [9]:
rag_prompt = hub.pull("rlm/rag-prompt")

#### Define the LLM

In [19]:
llm = OCIGenAILLM(
    temperature=0.5, config=config, compartment_id=COMPARTMENT_OCID, endpoint=ENDPOINT
)

#### Define the (Lang)Chain

In [20]:
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | rag_prompt | llm

#### Process the question

In [26]:
QUESTION = (
    """Chi è il responsabile della Funzione RGS Sistemi ERP? Rispondi in italiano"""
)

In [27]:
response = rag_chain.invoke(QUESTION)

print(response)

 La Funzione RGS Sistemi ERP è affidata a Gianna Caralla.
La Direzione Cloud Data Center, la cui responsabilità è affidata a Luca Bargellini, cura lo sviluppo e l’evoluzione dell’infrastruttura tecnologica e di sicurezza per la realizzazione e l’erogazione dei servizi e delle soluzioni IT a supporto degli ecosistemi di business.
La Direzione si articola in:
Funzione Change & Service Management, affidata a Giuseppe Spoto, assicura le attività di governo di change, release e configuration management.
Pag. 3 di 5
La Funzione si articola nelle seguenti strutture:
\uf0b7 Unità Organizzativa Change & Release Management Responsabile: Francesco Milone;
\uf0b7 Unità Organizzativa Configuration Management Responsabile: Sebastiano Luciano;
\uf02d Funzione Technical System Management, affidata a Luca Milanesi, assicura l’implementazione e il tuning dei sistemi hardware e software.
Pag. 4 di 5
La Funzione si articola nelle seguenti strutture:
\uf0b7 Unità Organizzativa System
